In [2]:
import numpy as np
import torch 
import matplotlib.pyplot as plt
import os
import re
import click
import tqdm
import pickle
import numpy as np
import torch
import PIL.Image
import dnnlib
from torch_utils import distributed as dist
from training.networks import SongUNet, EDMPrecond
# dist.init()
from generate import edm_sampler, ablation_sampler, StackedRandomGenerator

In [3]:
net_save='/data/edm_outputs/00000-FLAIR-uncond-ddpmpp-edm-gpus3-batch15-fp32-FLAIR_ddpm/network-snapshot-001206.pkl'
# net_save='/data/edm_outputs/00001-T2-uncond-ddpmpp-edm-gpus3-batch15-fp32-T2_ddpm/network-snapshot-001658.pkl'
# net_save='/data/edm_outputs/00002-T1-uncond-ddpmpp-edm-gpus3-batch15-fp32-T1_ddpm/network-snapshot-001005.pkl'

with dnnlib.util.open_url(net_save, verbose=(dist.get_rank() == 0)) as f:
        net = pickle.load(f)['ema']

In [4]:
class_labels = None
lat = torch.randn(1,2,640,320)
t = torch.tensor(1.)[None]
# # print(net.model.img_resolution)
# # print(net.in_channels)
# # print(net.out_channels)
# # print(net.label_dim)
# # print(net.augment_dim)
# # print(net.model_channels)
# # print(net.channel_mult)
denoised = net.model(lat, t, class_labels)
# print(net)

In [13]:
model_type='SongUNet'
embedding_type='positional'
encoder_type='standard'
decoder_type='standard'
channel_mult_noise=1
resample_filter=[1,1]
model_channels=128
channel_mult=[2,2,2]
img_resolution = 318
in_channels = 2
out_channels = 2

net1 = SongUNet(img_resolution=img_resolution, in_channels=in_channels,
                out_channels=out_channels, channel_mult=channel_mult)
edmNet = EDMPrecond(img_resolution=img_resolution,img_channels=in_channels,
                    label_dim = 0, model_type=model_type, channel_mult=channel_mult)

In [14]:
class_labels = None
lat = torch.randn(1,2,318,318)
t = torch.tensor(1.)
# print(t.shape)
# denoised = net1(lat, t[None], class_labels)
denoised_EDM = edmNet(lat,t,class_labels)

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 158 but got size 159 for tensor number 1 in the list.